In [ ]:
!pip install rdkit
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw, PandasTools, Descriptors, Descriptors3D, rdMolDescriptors, Scaffolds
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
PandasTools.RenderImagesInAllDataFrames(images = True) # to molecules visualization
from rdkit.Chem.Draw import IPythonConsole
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
foodb = pd.read_csv("https://raw.githubusercontent.com/DIFACQUIM/Food_chemicals_characterization/main/foodb_curated.csv")
fda = pd.read_csv("https://raw.githubusercontent.com/DIFACQUIM/Food_chemicals_characterization/main/fda_curated.csv")
unpda = pd.read_csv("https://raw.githubusercontent.com/DIFACQUIM/Food_chemicals_characterization/main/unpda_curated.csv")
purch = pd.read_csv("https://raw.githubusercontent.com/DIFACQUIM/Food_chemicals_characterization/main/purch_curated.csv")

In [ ]:
print(foodb.columns)
print(fda.columns)
print(unpda.columns)
print(purch.columns)

In [ ]:
foodb = foodb[['ID', "SMILES_chiral"]]
fda = fda[['ID', "SMILES_chiral"]]
unpda = unpda[['ID', "SMILES_chiral"]]
purch = purch[['ID', "SMILES_chiral"]]

In [ ]:
foodb.columns = ['ID', 'SMILES']
fda.columns = ['ID', 'SMILES']
unpda.columns = ['ID', 'SMILES']
purch.columns = ['ID', 'SMILES']

In [ ]:
# Specify the dataset each compound belongs to
foodb['DATASET'] = 'FooDB'
fda['DATASET'] = 'FDA'
unpda['DATASET'] = 'UNPD-A'
purch['DATASET'] = 'FooDB commercially available'

In [ ]:
# selection of a test database
"""
foodb = foodb.head(5)
fda = fda.head(5)
unpda = unpda.head(5)
purch = purch.head(5)
"""

In [ ]:
data = pd.concat([foodb, fda, unpda, purch], ignore_index = True)

In [ ]:
PandasTools.AddMoleculeColumnToFrame(data, smilesCol="SMILES",molCol='MOL')
PandasTools.AddMurckoToFrame(data, molCol='MOL', MurckoCol='SCAFFOLD')
PandasTools.AddMoleculeColumnToFrame(data, smilesCol="SCAFFOLD",molCol='SCA_MOL')
data

In [ ]:
# original databases are saved individually with their correspondant descriptors
foodb = data[data["DATASET"] == str('FooDB')]
fda = data[data["DATASET"] == str('FDA')]
unpda = data[data["DATASET"] == str('UNPD-A')]
purch = data[data["DATASET"] == str('FooDB commercially available')]

In [ ]:
foodb.to_csv('foodb_scaffolds.csv', index = False)
fda.to_csv('fda_scaffolds.csv', index = False)
unpda.to_csv('unpda_scaffolds.csv', index = False)
purch.to_csv('purch_scaffolds.csv', index = False)

In [ ]:
freq_foodb = foodb.groupby(['SCAFFOLD']).size()
print(f'FooDB data set had the next {freq_frag.shape[0]} different entries:', freq_frag.sort_values(ascending=False))
freq_fda = fda.groupby(['SCAFFOLD']).size()
print(f'FDA data set had the next {freq_fda.shape[0]} different entries:', freq_fda.sort_values(ascending=False))
freq_unpda = unpda.groupby(['SCAFFOLD']).size()
print(f'UNPD-A data set had the next {freq_unpda.shape[0]} different entries:', freq_unpda.sort_values(ascending=False))
freq_purch = purch.groupby(['SCAFFOLD']).size()
print(f'FooDB commercially available data set had the next {freq_frag.shape[0]} different entries:', freq_frag.sort_values(ascending=False))
freq_data = data.groupby(['SCAFFOLD']).size()
print(f'"data" data set had the next {freq_data.shape[0]} different entries:', freq_data.sort_values(ascending=False))

In [ ]:
foodb_sca = pd.DataFrame(freq_foodb, columns = ['AMOUNT'])
foodb_sca = foodb_sca.sort_values(by = 'AMOUNT', ascending = False)
foodb_sca = foodb_sca.reset_index(level = None, drop = False)
foodb_sca.at[0, 'SCAFFOLD'] = 'ACYCLIC' # this is needed when the data set present linear molecules (without scaffold)
print('QuimfraganDB:', foodb_sca.shape, foodb_sca.columns, foodb_sca)

fda_sca = pd.DataFrame(freq_fda, columns = ['AMOUNT'])
fda_sca = fda_sca.sort_values(by = 'AMOUNT', ascending = False)
fda_sca = fda_sca.reset_index(level = None, drop = False)
fda_sca.at[0, 'SCAFFOLD'] = 'ACYCLIC' # this is needed when the data set present linear molecules (without scaffold)
print('FDA:', fda_sca.shape, fda_sca.columns)

unpda_sca = pd.DataFrame(freq_unpda, columns = ['AMOUNT'])
unpda_sca = unpda_sca.sort_values(by = 'AMOUNT', ascending = False)
unpda_sca = unpda_sca.reset_index(level = None, drop = False)
unpda_sca.at[0, 'SCAFFOLD'] = 'ACYCLIC' # this is needed when the data set present linear molecules (without scaffold)
print('UNPD-A:', unpda_sca.shape, unpda_sca.columns)

purch_sca = pd.DataFrame(freq_purch, columns = ['AMOUNT'])
purch_sca = purch_sca.sort_values(by = 'AMOUNT', ascending = False)
purch_sca = purch_sca.reset_index(level = None, drop = False)
purch_sca.at[0, 'SCAFFOLD'] = 'ACYCLIC' # this is needed when the data set present linear molecules (without scaffold)
print('QuimfraganDB:', purch_sca.shape, purch_sca.columns)

data_sca = pd.DataFrame(freq_data, columns = ['AMOUNT'])
data_sca = data_sca.sort_values(by = 'AMOUNT', ascending = False)
data_sca = data_sca.reset_index(level = None, drop = False)
data_sca.at[0, 'SCAFFOLD'] = 'ACYCLIC' # this is needed when the data set present linear molecules (without scaffold)
print('DATA:', data_sca.shape, data_sca.columns, data_sca)

In [ ]:
print(f"The total amount of valid structures for scaffold computation in FooDB is: {sum(foodb_sca['AMOUNT'])}")
print(f"The total amount of valid structures for scaffold computation in FDA is: {sum(fda_sca['AMOUNT'])}")
print(f"The total amount of valid structures for scaffold computation in UNPD-A is: {sum(unpda_sca['AMOUNT'])}")
print(f"The total amount of valid structures for scaffold computation in FooDB commercially available is: {sum(purch_sca['AMOUNT'])}")
print(f"The total amount of valid structures for scaffold computation in 'data' data set is: {sum(data_sca['AMOUNT'])}")

foodb_sca

In [ ]:
foodb_sca['FREQUENCY'] = (foodb_sca['AMOUNT'] / sum(foodb_sca['AMOUNT'])) * 100
print(f"Total frequency in QuimfraganDB database: {sum(foodb_sca['FREQUENCY'])}, different entries: {sum(foodb_sca['AMOUNT'])}, different Murcko smiles:",
      len(foodb_sca['SCAFFOLD']))
fda_sca['FREQUENCY'] = (fda_sca['AMOUNT'] / sum(fda_sca['AMOUNT'])) * 100
print(f"Total frequency in FDA database: {sum(fda_sca['FREQUENCY'])}, different entries: {sum(fda_sca['AMOUNT'])}, different Murcko smiles:",
      len(fda_sca['SCAFFOLD']))
unpda_sca['FREQUENCY'] = (unpda_sca['AMOUNT'] / sum(unpda_sca['AMOUNT'])) * 100
print(f"Total frequency in UNPD-A database: {sum(unpda_sca['FREQUENCY'])}, different entries: {sum(unpda_sca['AMOUNT'])}, different Murcko smiles:",
      len(unpda_sca['SCAFFOLD']))
purch_sca['FREQUENCY'] = (purch_sca['AMOUNT'] / sum(purch_sca['AMOUNT'])) * 100
print(f"Total frequency in FooDB commercially available database: {sum(purch_sca['FREQUENCY'])}, different entries: {sum(purch_sca['AMOUNT'])}, different Murcko smiles:",
      len(purch_sca['SCAFFOLD']))
data_sca['FREQUENCY'] = (data_sca['AMOUNT'] / sum(data_sca['AMOUNT'])) * 100
print(f"Total frequency in 'DATA' database: {sum(data_sca['FREQUENCY'])}, different entries: {sum(data_sca['AMOUNT'])}, different Murcko smiles:",
      len(data_sca['SCAFFOLD']))

foodb_sca

### **Shannon Entropy Computation**
[Please check this video for more information](https://www.youtube.com/watch?v=2s3aJfRr9gE) \\
If SE=0, then all P compounds possess only a single cyclic system. If SE=log2n, then the P compounds are uniformly distributed among the n cyclic systems which represents maximum cyclic system diversity on the data set.

In [ ]:
# 'AMOUNT' and 'SCAFFOLD' are required to the computation, 'n' reffers to the quantity of scaffolds, and name refers to the name of the database to be computed
def SE (dataset, n, name):
  df = dataset[0:n]
  df['Pi'] = ((df['AMOUNT'])/sum(df['AMOUNT'])) # Compute relative frequency
  df['SE(i)'] = ((df['Pi']) * np.log2(df['Pi'])) # individual SE
  df['Shannon entropy'] = (-1 * df['SE(i)'])

  number = len(df['SCAFFOLD']) # amount of different scaffolds
  entropy = -sum(df['SE(i)']) # total SE of the dataset
  SSE = entropy / np.log2(number) # SE normalization

  M = sum(df['AMOUNT'])
  print(f'SE of dataset = {entropy}, SE[log2(n)] = {np.log2(number)}, SSE = {SSE}, n = {number}')
  print(f'The number of molecules considered in n = {M}')

  saving_path = f'shannon_{name}.csv'
  df.to_csv(saving_path, index = False)

  return df

In [ ]:
SE(dataset = data_sca, n = 15, name = 'data')

In [ ]:
SE(dataset = frag_sca, n = 15, name = 'FooDB')

In [ ]:
SE(dataset = fda_sca, n = 15, name = 'FDA')

In [ ]:
SE(dataset = unpda_sca, n = 15, name = 'UNPD-A')

In [ ]:
SE(dataset = unpda_sca, n = 15, name = 'FooDB commercially available')

In [ ]:
# Add a new column to identify the procedence of each scaffold
frag_sca['DATABASE'] = 'QuimfraganDB'
fda_sca['DATABASE'] = 'FDA'
unpda_sca['DATABASE'] = 'UNPD-A'
purch_sca['DATABASE'] = 'FooDB commercially available'

In [ ]:
# define a function which concatenate the colums 'AMOUNT' and 'FREQUENCY' in each dataframe

def concatenate(row):
  AMOUNT_str = str(int(row['AMOUNT']))
  FREQ_str = f"({row['FREQUENCY']:.2f}%)"
  return f"{row['DATABASE']} = {AMOUNT_str} {FREQ_str}"

In [ ]:
foodb_sca['LEGEND'] = foodb_sca.apply(concatenate, axis = 1)
fda_sca['LEGEND'] = fda_sca.apply(concatenate, axis = 1)
unpda_sca['LEGEND'] = unpda_sca.apply(concatenate, axis = 1)
purch_sca['LEGEND'] = purch_sca.apply(concatenate, axis = 1)

In [ ]:
foodb_sca.head(5)

In [ ]:
merged_1 = foodb_sca.merge(fda_sca[['SCAFFOLD', 'LEGEND']], on = 'SCAFFOLD', how = 'left', suffixes = ('', '_fda'))
merged_1['LEGEND_fda'].fillna('FDA = missing', inplace = True)
merged_2 = merged_1.merge(unpda_sca[['SCAFFOLD', 'LEGEND']], on = 'SCAFFOLD', how = 'left', suffixes = ('', '_unpda'))
merged_2['LEGEND_unpda'].fillna('UNPD-A = missing', inplace = True)
merged_3 = merged_2.merge(purch_sca[['SCAFFOLD', 'LEGEND']], on = 'SCAFFOLD', how = 'left', suffixes = ('', '_purch'))
merged_3['LEGEND_purch'].fillna('Purchasable = missing', inplace = True)

In [ ]:
merged_3.head(5)

In [ ]:
# define a function which concatenate the colums 'LEGEND...'

def concatenate_1(row):
  return f"{row['LEGEND']}\n{row['LEGEND_fda']}\n{row['LEGEND_unpda']}\n{row['LEGEND_purch']}"

In [ ]:
merged_3['LEGEND_final'] = merged_3.apply(concatenate_1, axis = 1)

In [ ]:
merged_3

In [ ]:
merged_3.to_csv('foodb_scaffolds_general.csv', index = False)